# Data Cleaning Process

## Beginning

Read in and investigate data. First, convert time to `pd.datetime` objects.

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import calendar
import numpy as np

df = pd.read_csv("data/final_data.csv")
df["trip_end_timestamp"] = pd.to_datetime(df["trip_end_timestamp"])
df["trip_start_timestamp"] = pd.to_datetime(df["trip_start_timestamp"])
df = df.drop(["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.1.1"], axis = 1)


In [2]:
df.shape

(119745, 14)

## Missing Values

Next, investigated missing values. Only trip seconds and trip miles have missing values, and relatively few.

Investigating the `trip_seconds` column, I can see that there are a few things. First, within the missing values there are three trips on November 11, 2018 and two trips on November 3, 2019 on which the end timestamp is before the trip start timestamp. All of the reamining missing data points come from June 6, 2019, and their trip endstamp happens exactly at midnight. There are 263 observations on that date, so the nulls represent ~60% of the data on that day.

Regarding the `trip_miles` column, there are only 3 missing values, with no discernable pattern.

For both of these, I dropped the missing data.

In [3]:
df.isnull().sum()

trip_start_timestamp           0
trip_end_timestamp             0
trip_seconds                 165
trip_miles                     3
pickup_community_area          0
dropoff_community_area         0
fare                           0
tip                            0
additional_charges             0
shared_trip_authorized         0
trips_pooled                   0
pickup_centroid_location       0
dropoff_centroid_location      0
date                           0
dtype: int64

In [4]:
df[df["trip_seconds"].isnull()]

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,shared_trip_authorized,trips_pooled,pickup_centroid_location,dropoff_centroid_location,date
900,2018-11-04 01:30:00,2018-11-04 01:15:00,NaN,16.281439,8,68,25.0,0,2.50,False,1,"{'type': 'Point', 'coordinates': [-87.63330803...","{'type': 'Point', 'coordinates': [-87.64249752...",2018-11-04
948,2018-11-04 02:00:00,2018-11-04 01:00:00,NaN,0.687300,8,8,2.5,0,2.50,False,1,"{'type': 'Point', 'coordinates': [-87.62910518...","{'type': 'Point', 'coordinates': [-87.63576009...",2018-11-04
1101,2018-11-04 01:45:00,2018-11-04 01:00:00,NaN,5.647486,6,77,12.5,0,2.50,False,1,"{'type': 'Point', 'coordinates': [-87.65156259...","{'type': 'Point', 'coordinates': [-87.66493772...",2018-11-04
57496,2019-06-06 10:30:00,2019-06-06 00:00:00,NaN,1.546445,6,7,5.0,0,2.55,False,1,"{'type': 'Point', 'coordinates': [-87.66126521...","{'type': 'Point', 'coordinates': [-87.65106188...",2019-06-06
57497,2019-06-06 16:00:00,2019-06-06 00:00:00,NaN,5.871260,22,32,15.0,0,2.55,False,1,"{'type': 'Point', 'coordinates': [-87.69453234...","{'type': 'Point', 'coordinates': [-87.63140652...",2019-06-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57753,2019-06-06 07:45:00,2019-06-06 00:00:00,NaN,12.951291,28,64,20.0,0,2.55,False,1,"{'type': 'Point', 'coordinates': [-87.64280846...","{'type': 'Point', 'coordinates': [-87.79124765...",2019-06-06
57754,2019-06-06 17:45:00,2019-06-06 00:00:00,NaN,3.875261,8,28,10.0,0,2.55,False,1,"{'type': 'Point', 'coordinates': [-87.63576009...","{'type': 'Point', 'coordinates': [-87.65723319...",2019-06-06
57757,2019-06-06 22:15:00,2019-06-06 00:00:00,NaN,1.833794,32,8,7.5,0,2.55,False,1,"{'type': 'Point', 'coordinates': [-87.63274648...","{'type': 'Point', 'coordinates': [-87.62076286...",2019-06-06
97018,2019-11-03 02:00:00,2019-11-03 01:15:00,NaN,4.585429,7,8,10.0,0,2.55,False,1,"{'type': 'Point', 'coordinates': [-87.66567683...","{'type': 'Point', 'coordinates': [-87.62887415...",2019-11-03


In [5]:
df[df["trip_miles"].isnull()]

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,shared_trip_authorized,trips_pooled,pickup_centroid_location,dropoff_centroid_location,date
62911,2019-06-25 19:30:00,2019-06-25 19:45:00,563.0,NaN,6,7,5.0,0,2.55,False,1,"{'type': 'Point', 'coordinates': [-87.64347095...","{'type': 'Point', 'coordinates': [-87.65400702...",2019-06-25
63984,2019-06-29 02:15:00,2019-06-29 02:15:00,513.0,NaN,24,8,10.0,0,2.85,False,1,"{'type': 'Point', 'coordinates': [-87.67531162...","{'type': 'Point', 'coordinates': [-87.64990722...",2019-06-29
71514,2019-08-28 18:45:00,2019-08-28 19:00:00,629.0,NaN,44,73,10.0,0,0.00,True,1,"{'type': 'Point', 'coordinates': [-87.61596952...","{'type': 'Point', 'coordinates': [-87.64889507...",2019-08-28


In [6]:
df = df.dropna()

## Sanity Check

Next, I wanted to conduct a sanity check to make sure every part of the data make sense. In the seconds, 77,160 seconds is the max, which is more than 21 hours. I investigated all of the rides within the data greater than 2 hours long (7200 seconds). Almost all of the observations made logical sense, except for the maximum number of seconds. I dropped this value. For the trip length (miles), none of the values looked suspicious.

In [7]:
df.describe()

,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trips_pooled
count,119577.000000,119577.000000,119577.000000,119577.000000,119577.000000,119577.000000,119577.000000,119577.000000
mean,981.809796,4.853748,24.485821,25.088152,10.057536,0.515291,2.786438,1.243876
std,721.880978,4.724699,19.284743,19.703073,7.417304,1.426151,1.673883,0.701467
min,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,510.000000,1.660458,8.000000,8.000000,5.000000,0.000000,2.550000,1.000000
50%,809.000000,3.217936,24.000000,24.000000,7.500000,0.000000,2.550000,1.000000
75%,1251.000000,6.168212,32.000000,32.000000,12.500000,0.000000,2.550000,1.000000
max,77160.000000,61.182788,77.000000,77.000000,225.000000,46.000000,21.550000,14.000000


In [8]:
df[df["trip_seconds"] > 7200]

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,shared_trip_authorized,trips_pooled,pickup_centroid_location,dropoff_centroid_location,date
4166,2018-11-16 17:30:00,2018-11-16 19:30:00,7309.0,27.641498,76,8,102.5,0,8.55,False,1,"{'type': 'Point', 'coordinates': [-87.90303966...","{'type': 'Point', 'coordinates': [-87.62621490...",2018-11-16
18757,2019-01-15 10:00:00,2019-01-16 07:30:00,77160.0,13.234300,24,8,7.5,1,0.67,False,1,"{'type': 'Point', 'coordinates': [-87.68957983...","{'type': 'Point', 'coordinates': [-87.63784420...",2019-01-15
32037,2019-03-04 10:30:00,2019-03-04 12:30:00,7848.0,13.626996,8,10,50.0,0,2.55,False,1,"{'type': 'Point', 'coordinates': [-87.63186394...","{'type': 'Point', 'coordinates': [-87.78301327...",2019-03-04
84676,2019-09-18 07:15:00,2019-09-18 09:15:00,7219.0,27.295204,76,8,57.5,6,8.30,False,1,"{'type': 'Point', 'coordinates': [-87.90303966...","{'type': 'Point', 'coordinates': [-87.62910518...",2019-09-18
96224,2019-11-01 15:30:00,2019-11-01 18:00:00,8719.0,40.090000,32,33,75.0,5,2.55,False,1,"{'type': 'Point', 'coordinates': [-87.62099291...","{'type': 'Point', 'coordinates': [-87.61735800...",2019-11-01
98399,2019-11-08 16:30:00,2019-11-08 19:15:00,10265.0,32.452849,31,31,75.0,0,7.75,False,1,"{'type': 'Point', 'coordinates': [-87.66352868...","{'type': 'Point', 'coordinates': [-87.66352868...",2019-11-08
118648,2020-01-27 09:15:00,2020-01-27 12:00:00,10320.0,2.369153,28,28,0.0,0,0.72,True,1,"{'type': 'Point', 'coordinates': [-87.68065411...","{'type': 'Point', 'coordinates': [-87.65723319...",2020-01-27
118880,2020-01-28 10:15:00,2020-01-28 19:45:00,33647.0,15.950901,32,28,225.0,0,5.39,False,1,"{'type': 'Point', 'coordinates': [-87.63274648...","{'type': 'Point', 'coordinates': [-87.64264899...",2020-01-28


In [9]:
df[df["trip_miles"] > 50]

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,shared_trip_authorized,trips_pooled,pickup_centroid_location,dropoff_centroid_location,date
50594,2019-05-11 18:00:00,2019-05-11 19:45:00,6492.0,60.673217,8,8,142.5,10,2.55,False,1,"{'type': 'Point', 'coordinates': [-87.61886835...","{'type': 'Point', 'coordinates': [-87.61886835...",2019-05-11
68672,2019-08-17 05:45:00,2019-08-17 06:45:00,3877.0,52.458358,76,76,62.5,0,7.55,False,1,"{'type': 'Point', 'coordinates': [-87.90303966...","{'type': 'Point', 'coordinates': [-87.90303966...",2019-08-17
78179,2019-07-23 23:30:00,2019-07-24 01:00:00,4846.0,50.455858,70,76,65.0,0,9.10,False,1,"{'type': 'Point', 'coordinates': [-87.70836570...","{'type': 'Point', 'coordinates': [-87.91362459...",2019-07-23
78737,2019-07-25 10:30:00,2019-07-25 12:00:00,5163.0,61.182788,76,32,75.0,13,9.80,False,1,"{'type': 'Point', 'coordinates': [-87.90303966...","{'type': 'Point', 'coordinates': [-87.63274648...",2019-07-25
103363,2019-11-27 15:45:00,2019-11-27 17:30:00,6252.0,51.632068,41,76,72.5,0,10.55,False,1,"{'type': 'Point', 'coordinates': [-87.59401544...","{'type': 'Point', 'coordinates': [-87.90303966...",2019-11-27
110180,2019-12-22 12:45:00,2019-12-22 14:45:00,6927.0,53.471446,35,56,57.5,9,7.55,True,3,"{'type': 'Point', 'coordinates': [-87.61083473...","{'type': 'Point', 'coordinates': [-87.75093428...",2019-12-22


In [10]:
df = df[df["trip_seconds"] < 70000]

## Data Processing

Next, I did some basic data processing. First, I read in the Community Areas file, and merged it twice with our DataFrame to get the names of the pickup and dropoff locations.


In [11]:
ca = pd.DataFrame(gpd.read_file("data/community_areas.geojson")).drop(["geometry"], axis = 1)
df = df.drop(["pickup_centroid_location", "dropoff_centroid_location"], axis = 1)
ca = ca[["community", "area_numbe"]]
ca["area_numbe"] = ca["area_numbe"].astype(int)
pickup_ca = ca.rename(columns={"area_numbe": "pickup_community_area", "community": "pickup_community_name"})
dropoff_ca = ca.rename(columns={"area_numbe": "dropoff_community_area", "community": "dropoff_community_name"})
merge_temp = pd.merge(df, pickup_ca, on = "pickup_community_area", validate = "m:1").drop(["pickup_community_area"], axis = 1)
final_df = pd.merge(merge_temp, dropoff_ca, on = "dropoff_community_area", validate = "m:1").drop(["dropoff_community_area"], axis = 1)

Then, I created time variables. I created a year, month, day, day of week, hour, and binary weekend indicator. NOTE that Friday is included in weekend here. Also note that I used the start trip timestamp to create these variables, because I figured that was more relevant from the perspective of the Uber driver.

In [12]:
final_df["year"] = final_df["trip_start_timestamp"].dt.year
final_df["month"] = final_df["trip_start_timestamp"].dt.month.apply(
    lambda x: calendar.month_name[x]
)
final_df["day"] = final_df["trip_start_timestamp"].dt.day
final_df["day_of_week"] = final_df["trip_start_timestamp"].dt.dayofweek
final_df["hour"] = final_df["trip_start_timestamp"].dt.hour
final_df["weekend"] = np.where(final_df["day_of_week"] >= 4, 1, 0)
final_df["day_of_week"] = final_df["day_of_week"].apply(
    lambda x: calendar.day_name[x]
)


Finally, I created an indicator variable for whether or not there was a tip, and then split into training, validation, and test.

In [13]:
final_df["nonzero_tip"] = np.where(final_df["tip"] > 0, 1, 0)
np.random.seed(3320)
train, validate, test = np.split(final_df.sample(frac=1), [int(.7*len(df)), int(.8*len(df))])